In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
from pathlib import Path
import numpy as np
import multiprocessing

In [5]:
# The optical flow input will look like this
of = torch.randn(3,2,640,480).cuda()

In [2]:
model = EfficientNet.from_pretrained("efficientnet-b0", in_channels=2, num_classes=1).cuda()

Loaded pretrained weights for efficientnet-b0


In [6]:
features = model(of)
features.shape

torch.Size([3, 1])

In [3]:
class OFDataset(Dataset):
    def __init__(self, of_dir, label_f):
        self.len = len(list(Path(of_dir).glob('*.npy')))
        self.of_dir = of_dir
        self.label_file = open(label_f).readlines()
    def __len__(self): return self.len
    def __getitem__(self, idx):
        of_array = np.load(Path(self.of_dir)/f'{idx}.npy')
        of_tensor = torch.squeeze(torch.Tensor(of_array))
        label = float(self.label_file[idx].split()[0])
        return [of_tensor, label]

In [4]:
ds = OFDataset('/home/sharif/Documents/RAFT/predictions', '/home/sharif/Documents/commai-challenge/data/labels/train.txt')

In [5]:
sample = ds[3]
assert type(sample[0]) == torch.Tensor
assert type(sample[1]) == float

In [6]:
cpu_cores = multiprocessing.cpu_count()
cpu_cores

6

In [7]:
dl = DataLoader(ds, batch_size=6, shuffle=True, num_workers=cpu_cores)

In [8]:
epochs = 10
steps = 25

In [9]:
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters())

In [10]:
writer = SummaryWriter('runs/optical_flow_exp_1')

In [ ]:
for epoch in range(epochs):
    
    running_loss = 0.0
    for i, sample in enumerate(dl):
        
        of_tensor = sample[0].cuda()
        label = sample[1].float().cuda()
        opt.zero_grad()
        pred = torch.squeeze(model(of_tensor))
        loss = criterion(pred, label)
        
        loss.backward()
        opt.step()
        
        running_loss += loss.item()
        if i % steps == 0: 
            writer.add_scalar('training loss',
                             running_loss / steps,
                             epoch * len(dl) + i)
            running_loss = 0.0